Author: Wolfgang Black <br>
Date: 2/7/25

## Prose Generation Coding Task

The challenge involves creating a backend system that transforms a series of bulleted "beats" into well-crafted prose that you would find in a novel. This tool is designed to aid writers in fleshing out as scene based on their outlines.

**Part 1: Beats-to-Prose**
Given a set of story beats, like this set for a sci-fi chapter:

1. Begin the chapter with Jack and Xander continuing their excavation on the lunar surface,
creating a sense of tension and anticipation.
2. Describe the barren landscape of the moon, emphasizing its desolation and the isolation
felt by Jack and Xander.
3. Use vivid and descriptive language to portray the moment when Jack and Xander uncover
an alien artifact, highlighting its mysterious and otherworldly appearance.
4. Convey Jack's excitement and trepidation as he realizes the significance of the discovery
and the potential implications it holds for humanity.
5. Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the
urgency of the situation.
6. Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship
and the gravity of the situation.
7. Highlight the importance of the discovery and the potential consequences it could have on
Earth and the lunar mining project.
8. Show Jack and Xander waiting anxiously for further instructions from Dr. Thorne, creating a
sense of anticipation and uncertainty.
9. Emphasize the isolation and vastness of the lunar landscape, reinforcing the challenges
and dangers that Jack and Xander face.
10. Use sensory details to immerse the reader in the lunar environment, such as the crunch of
lunar soil beneath their boots and the cold, airless atmosphere.
11. Portray Jack's determination and resolve to protect the discovery, even in the face of
potential conflict with rival miners.
12. Hint at the potential dangers and obstacles that Jack and Xander may encounter as they
navigate through a world of corporate greed and betrayal.
13. Foreshadow the secrets and mysteries that the alien artifact holds, building intrigue and
anticipation for future chapters.
14. End the chapter with a cliffhanger or unresolved tension, leaving the reader eager to
continue reading and discover what happens next.

Use these beats to produce ~1500 words of prose that matches these beats - in other words, if
a human were to summarize the prose, they would produce something along the lines of these
beats. It's okay to hallucinate details if they are missing from the beats in order to create the
narrative. Target approximately 100-150 words of output per beat. You are allowed to use any LLM or NLP techniques, including open source and closed source services like GPT or Claude.




In [1]:
# Test Input
TEST_BEATS_INPUT = [
    "Begin the chapter with Jack and Xander continuing their excavation on the lunar surface, creating a sense of tension and anticipation.",
    "Describe the barren landscape of the moon, emphasizing its desolation and the isolation felt by Jack and Xander.",
    "Use vivid and descriptive language to portray the moment when Jack and Xander uncover an alien artifact, highlighting its mysterious and otherworldly appearance.",
    "Convey Jack's excitement and trepidation as he realizes the significance of the discovery and the potential implications it holds for humanity.",
    "Show Jack immediately contacting Dr. Selene Thorne, emphasizing his trust in her and the urgency of the situation.",
    "Include dialogue between Jack and Dr. Thorne, showcasing their professional relationship and the gravity of the situation.",
    "Highlight the importance of the discovery and the potential consequences it could have on Earth and the lunar mining project.",
    "Show Jack and Xander waiting anxiously for further instructions from Dr. Thorne, creating a sense of anticipation and uncertainty.",
    "Emphasize the isolation and vastness of the lunar landscape, reinforcing the challenges and dangers that Jack and Xander face.",
    "Use sensory details to immerse the reader in the lunar environment, such as the crunch of lunar soil beneath their boots and the cold, airless atmosphere.",
    "Portray Jack's determination and resolve to protect the discovery, even in the face of potential conflict with rival miners.",
    "Hint at the potential dangers and obstacles that Jack and Xander may encounter as they navigate through a world of corporate greed and betrayal.",
    "Foreshadow the secrets and mysteries that the alien artifact holds, building intrigue and anticipation for future chapters.",
    "End the chapter with a cliffhanger or unresolved tension, leaving the reader eager to continue reading and discover what happens next."
]

## Solution to part I

To solve this I created a mulitagenic workflow. I decided to utilize LLMs for the majority of this work, using OpenAI's gpt-3.5-turbo for this demo. I've been experimenting with agents and passing data through LLMs recently and wanted to try this instead of more traditional approaches. 

Multi-Agentic Workflow: 
1. We initialize `BeatToStory()` - which is the pipeline class containing our agents.
2. The story beats, provided by the user, are passed into the BeatToStory instance 
3. We run `_.setup_pipe` which initializes the agents for the BeatToStory instance
    - if a list is not provided, we assume a pipeline of `[ContextAgent, ProseAgent, StoryAgent, LengthAgent, FlowAgent]`
4. `ContextAgent` reads through the beats, trying to identify key information, characters, setting, etc
5. `ProseAgent` reads in the beats and the context linking two beats and writes prose to narratively connect the beats while maintaining context
6. `StoryAgent` reviews the context and prose to verify that all the information was maintained and captured by the prose
7. `LengthAgent` reviews the prose length and verifies we're within our `[min, max]` word count per prose
8. `FlowAgent` acts as a final editor, rewriting the overall prose to prevent grammar or repetive language

the `BeatToStory()` class also has methods to disclose cost of each agent (based on shared token costs) and whether each step of the prose was successful. 

Below we can see an example of how well it works. 

In [2]:
# Note: OpenAI init was abstracted into the utils. please set an environment variable OPENAI_KEY
# to be accessible by `os.environ.get('OPENAI_KEY')`
import sys
sys.path.append('..') #only needed in notebook
from utils import BeatToStory
beatbot = BeatToStory()

In [3]:
beatbot.beats = TEST_BEATS_INPUT
beatbot.setup_pipeline()
print(beatbot.describe_pipeline())

ContextAgent
 llm: gpt-3.5-turbo -Note: Only GPT support atm 
 Agentic Prompt:You are ContextAgent. Extract key scene details and validate physics/environment.
        Before generating JSON output:
        1. Check environmental consistency: Temperature, atmosphere, gravity, sound propagation
        2. Validate technology limitations and capabilities

        Return JSON with:
        1. setting: {
            location: current scene location,
            location_change: true only if location explicitly changes,
            important_details: key events, mood, or setting details
        }
        2. characters: list of characters with their status {
            name: character name,
            character_location: "on stage" or "off stage",
            status_change: true if role/presence changes
        }
        If no previous context, set change flags to false.


ProseAgent
 llm: gpt-3.5-turbo -Note: Only GPT support atm 
 Agentic Prompt:You are ProseAgent, a creative writing ass

In [4]:
beatbot.pipe()
print(beatbot.story_length)
print(beatbot.pipeline_cost())
print(beatbot.edited_story)

1492
{'context': 0.006312, 'prose': 0.013845000000000003, 'story': 0.004294000000000001, 'length': 0.0, 'flow': 0.003926, 'total': 0.028377000000000003}
Jack and Xander ventured deeper into the lunar surface, the desolation of the barren landscape weighing heavily on them. The vast emptiness surrounding them amplified their isolation, adding an uneasy layer to their excavation. The silence of space enveloped them, broken only by the sound of their tools against the rocky terrain. Despite their shared mission, each step seemed to widen the gap between them, the tension growing palpable with every passing moment. Jack's gaze flickered towards the horizon, a subtle movement not escaping Xander's notice, deepening the unspoken apprehension between the two lunar explorers.

Their tools scraped against the lunar surface, revealing a glint of unearthly light that caught their eyes. With cautious excitement, they uncovered an alien artifact buried beneath the moon's dust. Its surface shimmered

In [5]:
for i in range(20):
    beatbot.pipe()
    print(f"beatbot story gen {i}")
    print(f"length of edited story: {beatbot.story_length}, and total cost: ${round(beatbot.pipeline_cost()['total'],2)}")
    print('-'*50+'\n')

beatbot story gen 0
length of edited story: 1492, and total cost: $0.03
--------------------------------------------------

beatbot story gen 1
length of edited story: 1505, and total cost: $0.04
--------------------------------------------------

beatbot story gen 2
length of edited story: 1495, and total cost: $0.04
--------------------------------------------------

beatbot story gen 3
length of edited story: 1493, and total cost: $0.04
--------------------------------------------------

beatbot story gen 4
length of edited story: 1493, and total cost: $0.05
--------------------------------------------------

beatbot story gen 5
length of edited story: 1494, and total cost: $0.05
--------------------------------------------------

beatbot story gen 6
length of edited story: 1503, and total cost: $0.06
--------------------------------------------------

beatbot story gen 7
length of edited story: 1493, and total cost: $0.06
--------------------------------------------------

beatbot 

KeyboardInterrupt: 

### **Part 2: Incorporating Story Metadata**

Now we want to extend the functionality of Beats-to-Prose to accept a few new parameters:
- a list of characters (name and details about them)
- setting
- genre
- style of prose and have that alter the generated prose.

Describe in detail your approach to this. Some things to consider:

1. What models would you use? Why?
2. If you need to procure data, how would go about doing that? How would you prepare the data?
3. What is your approach to producing prose that matches the parameters?

How would you evaluate whether the generations are good quality and fit the parameters specified?
Consider cost and latency considerations

## Solution to Part II

For this I'll further evolve the architecture from part I. Because I have an agentic workflow I can add additional agents and checks to the code to the pipeline. This allows me flexibility to help change style, affect prose, get additional reference material (not yet supported), etc. I could even use the internet, forums, or RAG to help guide prose generation. 

For this part I'll add 2 main agents.
- MetadataAgent: This agent takes in metadata provided by the user and merges it with the context generated by ContextAgent at each beat. This can help provide general character motivations, scene details, etc. Like LengthAgent - this is not an LLM based agent and therefor does not increase token cost. For this problem I expected a specific format for the user provided metadata, however that can be handled with a little client training and ETL pipeline. 
- StoryGenreAgent: This agent is an aggressive editor dedicated to rewriting prose in a specific style or genre. This is currently my weakest solution - It would likely be better split into multiple agents with unique specific prompts as opposed to just formatting the genre/style into the Agent Prompt. However, I wanted to experiment with general prompts. One can see this does add airs of genre to the prose. However style is specifically difficult. 

Aside: for me, I think of style as poetic, novel, newspaper, talk like a pirate. Happy to talk details in the conversation. 

As for how I'd capture this data if it wasn't provided by the user. We can scan context from previous writings, chapters, forums/sites, styles and ask contextAgent to further evolve its context to include things like setting details (Where is this setting, what happens there, who lives there, etc) and character details (who is this person, who are they related too, what motivates them). However, I did that partially for part I - so expanding that wouldn't be much work. Here I wanted to show off how incorporating the writers specific vision for a setting/character would enhance the story. 

Other honorable mentions: 
- RAG: use for data/context gathering. Could submit tomes of information on worlds, characters, race relations (Tolkien like - dwarves and elves), and have context generate details based on expected tropes and source material. 
- Model finetuning: We can finetune open source models using SST or we can finetune chatgpt using their API - but these finetune models can specialize in writing specific styles or translating prose from one standard into another. In my experience this is significantly stronger - but does require more testing and data. 
- For data capture for finetuning we could use NLP techniques like semantic search, similarity, and classification to get data for finetuning on style, genre, or event specific models

For cost, we're already reducing token generation - though finetuning local models/open source and doing distilation training can drive costs down in the long run. 

For metrics/generational goodness this is a BIG question in the field. Some things we could explore are topic adherence (similar to my context/StoryAgents) paired with keyword analysis and semantic similarity. For instance, we can search for a Town, a Character, or an Event and make sure descriptions/events/etc are all similar around that keyword. Though this would have to be researched for long term viability. For creative quality, you could use perplexity scores, coherence metrics, and readability measures to assess the prose's fluency and engagement level.

I also like to include human-in-the-loop metrics. We could pass prose off to editors, lay people, writers, etc to review. Some passages would be human written, some would be AI. We'd use scores of readability, consistency, flow, etc from the audience to assess goodness. Once we have this dataset, we could train NLP classifiers. 


For an example of how these systems may all eventually perform, see my demo'd BeatToStory class incorproate Metadata and genre/style from writer (user) specified metadata

In [6]:
from utils import ContextAgent, MetadataAgent, ProseAgent, StyleGenreAgent, StoryAgent, LengthAgent, FlowAgent

## User Metadata Update Only

In [7]:
metabot = BeatToStory()
metabot.beats = TEST_BEATS_INPUT
metabot.user_metadata = {"setting":
        {"location": "lunar surface",
         "notes": "The Jack and Xander are alone. Dr. Thorne is in her lab. They are on the dark side of the moon",
        },
    "characters": [
        {"name": "Jack", "profile": "An astronaut miner working on the moon. An educated adventure, he's working alongside Xander and Dr. Thorne searching the moon for precious metals. He's currently contracted to the Corp (a big pharma corp), but has a distrust of large companies. His main goal is to prevent the privatization of data that could help humanity. He's our main character loosely modeled off of Indian Jones"},
        {"name": "Xander", "profile": "An astronaught miner working alongside Jack on the moon. Xander served in the British Royal Space Force and bounced around from career to career. He's got serious debts and a doubious history... Is he really trust worthy? He met Jack during one of Jacks expidition to the Sahara, where Jack saved his life. He's worked along side Jack any time Jacks needed support flying - whether that's terrestrial or beyond"},
        {"name": "Dr. Selene Thorne", "Profile": "Selene Thorne is a brilliant Astrobiophysics. During her graduate program she discovered frequencies from deepspace that seemed to be bouncing off the moon. These frequencies seemingly contained the genetic code of the human race - and were set on repeat. Her lifes work has been to find some sign of those frequecies, how they're being broadcast, and from where. She's worked closely with Jack behind the scenes, trying to prevent evil pharam corps from privatizing the data and charging for the amazing potential medical advances. She has some private shared history with Jack - but the only sign of it in their interactions is deep trust."}]}

metabot.agents = {
                "context": ContextAgent(),
                "meta": MetadataAgent(),
                "prose": ProseAgent(min_words=metabot.min_words_per_beat, 
                             max_words=metabot.max_words_per_beat),
                "story": StoryAgent(),
                "length": LengthAgent(min_words=metabot.min_words_per_beat, 
                                    max_words=metabot.max_words_per_beat),
                "flow": FlowAgent()
            }
metabot.pipe()
print(metabot.story_length)
print(metabot.pipeline_cost())
print(metabot.edited_story)


1389
{'context': 0.0070420000000000005, 'meta': 0.0, 'prose': 0.0104065, 'story': 0.0023165, 'length': 0.0, 'flow': 0.0036965, 'total': 0.023461500000000003}
As Jack and Xander ventured deeper into the lunar surface, an undeniable tension brewed between them, echoing the desolation of the barren moon that surrounded them. The vast emptiness accentuated their isolation, underscoring the gravity of their mission. Jack's determined strides kicked up lunar dust, starkly contrasting with the silent void enveloping them. Meanwhile, Xander's gaze swept over the rugged landscape, momentarily eclipsing his troubled past in the face of the lunar expanse. The excavation resounded with the rhythmic clinks of their tools against the rocky terrain, a relentless reminder of their pursuit amidst the lunar solitude.

Their excavation led them to a concealed chamber where a glint of alien metal seized their attention. With cautious hands, they unveiled an artifact unlike anything seen on Earth. Its surf

## Genre/Style Updaet

In [8]:
metastylebot = BeatToStory()
metastylebot.beats = TEST_BEATS_INPUT
metastylebot.user_metadata = {"setting":
        {"location": "lunar surface",
         "notes": "The Jack and Xander are alone. Dr. Thorne is in her lab. They are on the dark side of the moon",
        },
    "genre": "horror",
    "style": "newspaper", 
    "characters": [
        {"name": "Jack", "profile": "An astronaut miner working on the moon. An educated adventure, he's working alongside Xander and Dr. Thorne searching the moon for precious metals. He's currently contracted to the Corp (a big pharma corp), but has a distrust of large companies. His main goal is to prevent the privatization of data that could help humanity. He's our main character loosely modeled off of Indian Jones"},
        {"name": "Xander", "profile": "An astronaught miner working alongside Jack on the moon. Xander served in the British Royal Space Force and bounced around from career to career. He's got serious debts and a doubious history... Is he really trust worthy? He met Jack during one of Jacks expidition to the Sahara, where Jack saved his life. He's worked along side Jack any time Jacks needed support flying - whether that's terrestrial or beyond"},
        {"name": "Dr. Selene Thorne", "Profile": "Selene Thorne is a brilliant Astrobiophysics. During her graduate program she discovered frequencies from deepspace that seemed to be bouncing off the moon. These frequencies seemingly contained the genetic code of the human race - and were set on repeat. Her lifes work has been to find some sign of those frequecies, how they're being broadcast, and from where. She's worked closely with Jack behind the scenes, trying to prevent evil pharam corps from privatizing the data and charging for the amazing potential medical advances. She has some private shared history with Jack - but the only sign of it in their interactions is deep trust."}]}

metastylebot.genre = metastylebot.user_metadata['genre']
metastylebot.style = metastylebot.user_metadata['style']
metastylebot.agents = {
                "context": ContextAgent(),
                "meta": MetadataAgent(),
                "prose": ProseAgent(min_words=metastylebot.min_words_per_beat, 
                             max_words=metastylebot.max_words_per_beat),
                "genre": StyleGenreAgent(style_guide = metastylebot.user_metadata['genre']),
                "style": StyleGenreAgent(style_guide = metastylebot.user_metadata['style']),
                "story": StoryAgent(),
                "length": LengthAgent(min_words=metastylebot.min_words_per_beat, 
                                    max_words=metastylebot.max_words_per_beat),
                "flow": FlowAgent()
            }

metastylebot.pipe()
print(metastylebot.story_length)
print(metastylebot.pipeline_cost())
print(metastylebot.edited_story)

1320
{'context': 0.006312, 'meta': 0.0, 'prose': 0.018109999999999998, 'genre': 0.0103795, 'style': 0.010087, 'story': 0.004055999999999999, 'length': 0.0, 'flow': 0.0036360000000000003, 'total': 0.052580499999999995}
Lunar Nightmare Unfolds for Jack and Xander

Jack and Xander's lunar excavation took a chilling turn, the sharp clang of their tools piercing the eerie silence of the desolate lunar landscape. As they delved deeper into the moon's depths, a sense of dread gripped them tighter, the gravity of their task bearing down like a heavy cloak. Each discovery sent a malevolent energy crackling through the air, fueling their fixation. Within the vast emptiness, a sinister presence strained their partnership, forging a malevolent alliance amidst the unforgiving lunar terrain.

Venturing further into the barren lunar expanse, Jack and Xander's eyes were ensnared by an eerie light. With held breath, they uncovered a sinister artifact buried beneath the desolate soil. Its surface glowed

## **Part 3: Describe how you might turn this code into an API**

Imagine you have a working protoype and would like the application team to start using your solution. You can assume you are using the FastAPI framework or something similar. You can use psuedocode for this exercise.

1. What would a hander look like, and what parameters would it receive?

In [8]:
# Write your solution for the handler here You don't have to run this cell
## In my solutions I use pydantic classes to validate the input and output of the handler. This helps ensure requests match my specified schema and that I return the correct response format.
## If a failure hits BEFORE this, I'll get a 400 error before it hits my business logic.
#  If a failure hits AFTER this, I'll get a 500 error. This is a good way to ensure that the handler is working as expected.
# I also ensure the output is standardized and easy to utilize downstream.
## This is my main.py file - this is the main file that'd be run by the Docker container. 

